In [5]:
using DelimitedFiles
using Distances
using StatsBase 
using Random
using Measures
using Statistics
using LinearAlgebra
using Distributions
using Plots
using Printf
rng = Random.MersenneTwister(1234);
include("../src/OU/Bezier_interpolation.jl")
include("../src/OU/analysis.jl")
include("../src/common/common_Bezier.jl");
include("../src/common/common_analysis.jl");

## Set parameters

In [22]:
P = 40 
L = 50 
dt = 0.001
D = 0.1
sqrt_Ddt = sqrt(D*dt)
T = 1000;

dir_out = "../out/OU/";

In [23]:
# Get interaction matrix
(patterns_embbeded, J) = get_patterns_J(L,P); 

# Generate trajecotries and energy of the Ornstein-Uhlenbeck process

In [24]:
n_ensemble_temp=100
T_temp = 10000

x_ensembles_temp = zeros(n_ensemble_temp, T_temp, L)
n_t_observe_temp = 5
t_ensembles_temp = []

x0 = zeros(L)
for i in 1:(Int(floor(0.3*L)))
    x0[i] = 5
end
for i in (Int(floor(0.7*L))):L
    x0[i] = -5
end

n_t_count_max = 1
for n in 1:n_ensemble_temp
    x = copy(x0)
    n_t_count = 1
    for n_t in 0:T_temp
        #x += dt * ( J*copy(x) + sqrt_Ddt*randn(L) )  
        x += dt * J*copy(x) +sqrt_Ddt* randn(L) 
        if(n_t%n_t_observe_temp==0)
            x_ensembles_temp[n, n_t_count, :] = copy(x)
            if(n==1)
                t_ensembles_temp = push!(t_ensembles_temp, n_t * dt)
            end
            n_t_count +=1
        end
    end
    if(n==1)
        n_t_count_max = n_t_count-1
    end
end
erg_evolution_avg = [mean([get_energy(x_ensembles_temp[n, n_t_count,:], J) for n in 1:n_ensemble_temp]) for n_t_count in 1:n_t_count_max];


In [25]:
# Get effective energy of OU processes
n_ensemble_temp=100
T_temp = 10000
n_t_observe_temp = 5
erg_evolution_avg = get_effective_energy_OU(n_ensemble_temp, n_t_observe_temp, T_temp, L, J, sqrt_Ddt);

In [26]:
# Out put: whole trajectories and energy profiles
fout_trajectory = open(dir_out * "trajectory.dat", "w")
for n in 1:n_ensemble_temp
    for n_t in 1:n_t_count_max
        print(fout_trajectory, n, "\t",n_t, "\t", n_t*dt, "\t")
        for i in 1:L
            print(fout_trajectory, x_ensembles_temp[n,n_t,i]," ")
        end
        println(fout_trajectory, "\n")
    end
end
close(fout_trajectory)

# out put energy trajectory to a file
fout_energy = open(dir_out * "energy.dat", "w")
for n in 1:size(erg_evolution_avg,1)
    println(fout_energy, n, "\t", n*dt, "\t", erg_evolution_avg[n])
end
close(fout_energy);


fout_times = open(dir_out * "tiems.dat", "w")
for t in t_ensembles_temp
    println(fout_times, t)
end
close(fout_times);


# Infer interaction coupling parameter 

In [27]:
# set sampling interval
n_ensemble=500 #--- original
T = 10000
(x_ensembles, t_ensembles) = get_x_t_ensembles(L, T, dt, D, n_ensemble, J);

## Invese of covariance, a simple equilibrium inference method based on the mean-field theory. 

In [28]:
t_observe = 3000
C_set = []
c_tot = zeros(L,L)

index_observe = collect(0:T)
index_observe = index_observe[ index_observe .% t_observe .== 0] .+1

x_ensembles_partial = copy(x_ensembles[:,index_observe,:])
t_ensembles_partial = copy(t_ensembles[index_observe])
K = size(index_observe,1)
for k in 1:(K-1)
    x_set_k = x_ensembles_partial[:,k,:]
    x_mean = sum(x_set_k, dims=1) / size(x_ensembles_partial,1)
    c_k = x_set_k' * x_set_k / size(x_ensembles_partial,1) - x_mean' * x_mean;
    c_tot += copy(c_k)
end
J_est_eq = -inv(cholesky(c_tot / (K-1) + 0.01*I));

## Naive MPL: MPL with the pice-wise constant interpolation

In [29]:
my_numerator = zeros(L,L)
my_denominator = zeros(L,L);

for n in 1:n_ensemble
    for k in 1:(K-1)
        dx = x_ensembles_partial[n, k+1,:] - x_ensembles_partial[n, k,:]
        dt_int = t_ensembles_partial[k+1] - t_ensembles_partial[k]
        my_numerator += copy( x_ensembles_partial[n, k,:] * dx' ) 
        my_denominator += copy( dt_int * x_ensembles_partial[n, k,:]*x_ensembles_partial[n, k,:]' )
    end
end
J_est = (my_denominator + I) \  my_numerator ;

## Linear MPL: MPL with the linear interpolation

In [30]:
Q1 = [1/3 1/6; 1/6 1/3];
my_numerator_Lin = zeros(L,L)
my_denominator_Lin = zeros(L,L);

for n in 1:n_ensemble
    for k in 1:(K-1)
        dt_int = t_ensembles_partial[k+1] - t_ensembles_partial[k]        
        x_k = copy(x_ensembles_partial[n,k,:])
        x_kp1 = copy(x_ensembles_partial[n,k+1,:])
        
        ##### integrated outer product of xΔx^T is (x(1)+x(0))*(x(1)-x(0))^T
        my_numerator_Lin += copy(0.5 * (x_kp1+x_k)*(x_kp1-x_k)' ) 
        ##### integrated covariance #####
        my_denominator_Lin += copy( dt_int * x_ensembles_partial[n, k:(k+1),:]' * Q1 * x_ensembles_partial[n, k:(k+1),:] )
    end
end
J_est_Lin = (my_denominator_Lin + I) \  my_numerator_Lin ;

## Bezier MPL: MPL with Bezier interpolation

In [31]:
# The following shuld be included in protected scripts.

## This metrix characterizes the x_Bez and x_Bez product
Mat_Bez = get_integrated_Bezier_2nd();
## This matrix is for the x_Bez and d/dt x_Bez product
Mat_Bez_2 = get_integrated_Bezier_single();


In [32]:
my_numerator_Bez = zeros(L,L)
my_denominator_Bez = zeros(L,L);

for n in 1:n_ensemble
    #estimating the controle point vectos a and b.
    a_vec_set = zeros(K-1, L)
    b_vec_set = zeros(K-1, L)
    for i in 1:L
        (a_vec, b_vec) = get_a_b_Bezier(K-1, x_ensembles_partial[n,:,i])
        a_vec_set[:, i] = copy(a_vec)
        b_vec_set[:, i] = copy(b_vec)    
    end
    
    for k in 1:(K-1)
        dt_int = t_ensembles_partial[k+1] - t_ensembles_partial[k]
        
        #----- set x_Bez matrix ------#
        x_Bez = zeros(L, 4)
        x_Bez[:, 1] = copy(x_ensembles_partial[n,k,:])
        x_Bez[:, 2] = 3*copy(a_vec_set[k, :]) # coefficient 3 should be carefully treated!
        x_Bez[:, 3] = 3*copy(b_vec_set[k, :])
        x_Bez[:, 4] = copy(x_ensembles_partial[n,k+1,:]);
        
        #------- x_Bez and x_Bez product for the denominator -------#
        x_xT_Bez = x_Bez * Mat_Bez * x_Bez';
        #------- x_Bez and d/dt x_Bez product for the numerator -------#
        x_dxT_Bez = (x_Bez * Mat_Bez_2 * x_Bez')';
        
        ##### integrated outer product of xΔx^T is (x(1)+x(0))*(x(1)-x(0))^T
        my_numerator_Bez += copy( x_dxT_Bez ) 
        ##### integrated covariance #####
        my_denominator_Bez += copy( dt_int * x_xT_Bez )
    end
end
J_est_Bez = (my_denominator_Bez + I) \  my_numerator_Bez ;

## Output coupling matrices 

In [33]:
fout_Jtrue = open(dir_out * "coupling_true.dat", "w") 
fout_J_inv_cov = open(dir_out * "coupling_inverse-cov.dat", "w") 
fout_J_naive = open(dir_out * "coupling_naive-MPL.dat", "w") 
fout_J_lin = open(dir_out * "coupling_linear-MPL.dat", "w") 
fout_J_Bez = open(dir_out * "coupling_Bezier-MPL.dat", "w"); 

for i in 1:L
    for j in i:L
        println(fout_Jtrue, i, " ", j, " ", J[i,j])
        println(fout_J_inv_cov, i, " ", j, " ", J_est_eq[i,j])
        println(fout_J_naive, i, " ", j, " ", J_est[i,j])        
        println(fout_J_lin, i, " ", j, " ", J_est_Lin[i,j])        
        println(fout_J_Bez, i, " ", j, " ", J_est_Bez[i,j])        
    end
end
close(fout_Jtrue); close(fout_J_inv_cov); close(fout_J_naive); close(fout_J_lin); close(fout_J_Bez);


# Analysis: sampling interval dependency

In [34]:
# This process takes 50-150 sec.
t_observe_set = [1000, 2000, 3000, 4000, 5000] 

n_observe_set = size(t_observe_set,1)

n_iteration_max = 10
@printf("# of iteration and sample size are %d and %d, respectively\n", n_iteration_max, n_ensemble)
@printf("D, diffusion is %.3f\n", D)
@printf("dt, discritized time unit is %.3f\n", dt)
corr_ensemble_Lin = zeros(n_observe_set, 2, 2, n_iteration_max)# Ps, diag/offdiag, slope/corr, n
corr_ensemble_Bez = zeros(n_observe_set, 2, 2, n_iteration_max) 
id_slop, id_cor=1,2
id_diag, id_offdia=1,2

erg_evolution_avg_set = []

@time for n_itr in 1:n_iteration_max
    
    (patterns_embbeded, J) = get_patterns_J(L,P);
    (x_ensembles, t_ensembles) = get_x_t_ensembles(L, T, dt, D, n_ensemble, J);
    
    for id_observe in 1:n_observe_set
        t_observe = t_observe_set[id_observe]
        index_observe = collect(0:T)
        index_observe = index_observe[ index_observe .% t_observe .== 0] .+ 1    
        
        x_ensembles_partial = copy(x_ensembles[:,index_observe,:])
        t_ensembles_partial = copy(t_ensembles[index_observe]) 
        K = size(index_observe,1)
        #@show L, K

        J_est_Lin = estimate_Linear(L, x_ensembles_partial, t_ensembles_partial )
        J_est_Bez = estimate_Bezier(L, x_ensembles_partial, t_ensembles_partial )
        (J_diag, J_offdiag) = get_diag_offdiag(J)
        (J_Lin_diag, J_Lin_offdiag) = get_diag_offdiag(J_est_Lin)
        (J_Bez_diag, J_Bez_offdiag) = get_diag_offdiag(J_est_Bez)

        (slope_diag_Lin, cor_diag_Lin) = get_corslope(J_diag, J_Lin_diag)
        (slope_offdiag_Lin, cor_offdiag_Lin) = get_corslope(vec(J_offdiag), vec(J_Lin_offdiag))
        (slope_diag_Bez, cor_diag_Bez) = get_corslope(J_diag, J_Bez_diag )
        (slope_offdiag_Bez, cor_offdiag_Bez) = get_corslope(vec(J_offdiag), vec(J_Bez_offdiag) )

        corr_ensemble_Lin[id_observe, id_diag, id_slop, n_itr] = slope_diag_Lin
        corr_ensemble_Lin[id_observe, id_diag, id_cor, n_itr] = cor_diag_Lin
        corr_ensemble_Lin[id_observe, id_offdia, id_slop, n_itr] = slope_offdiag_Lin
        corr_ensemble_Lin[id_observe, id_offdia, id_cor, n_itr] = cor_offdiag_Lin

        corr_ensemble_Bez[id_observe, id_diag, id_slop, n_itr] = slope_diag_Bez
        corr_ensemble_Bez[id_observe, id_diag, id_cor, n_itr] = cor_diag_Bez
        corr_ensemble_Bez[id_observe, id_offdia, id_slop, n_itr] = slope_offdiag_Bez
        corr_ensemble_Bez[id_observe, id_offdia, id_cor, n_itr] = cor_offdiag_Bez
    end
end

# of iteration and sample size are 10 and 500, respectively
D, diffusion is 0.100
dt, discritized time unit is 0.001
249.942151 seconds (598.48 M allocations: 1.123 TiB, 30.67% gc time)


In [35]:
# Compute Pearson correlation and slope between true and estimated coupling matrices 

corr_diag_slope_Lin = [mean(corr_ensemble_Lin[id_observe, id_diag, id_slop,:]) for id_observe in 1:n_observe_set]
corr_diag_slope_Bez = [mean(corr_ensemble_Bez[id_observe, id_diag, id_slop,:]) for id_observe in 1:n_observe_set]
corr_offdiag_slope_Lin = [mean(corr_ensemble_Lin[id_observe, id_offdia, id_slop,:]) for id_observe in 1:n_observe_set]
corr_offdiag_slope_Bez = [mean(corr_ensemble_Bez[id_observe, id_offdia, id_slop,:]) for id_observe in 1:n_observe_set];

sqrt_n_iteration_max = sqrt(n_iteration_max)
corr_diag_slope_Lin_std = [std(corr_ensemble_Lin[id_observe, id_diag, id_slop,:])/sqrt_n_iteration_max for id_observe in 1:n_observe_set]
corr_diag_slope_Bez_std = [std(corr_ensemble_Bez[id_observe, id_diag, id_slop,:])/sqrt_n_iteration_max for id_observe in 1:n_observe_set]
corr_offdiag_slope_Lin_std = [std(corr_ensemble_Lin[id_observe, id_offdia, id_slop,:])/sqrt_n_iteration_max for id_observe in 1:n_observe_set]
corr_offdiag_slope_Bez_std = [std(corr_ensemble_Bez[id_observe, id_offdia, id_slop,:])/sqrt_n_iteration_max for id_observe in 1:n_observe_set];

corr_diag_cor_Lin = [mean(corr_ensemble_Lin[id_observe, id_diag, id_cor,:]) for id_observe in 1:n_observe_set]
corr_diag_cor_Bez = [mean(corr_ensemble_Bez[id_observe, id_diag, id_cor,:]) for id_observe in 1:n_observe_set]
corr_offdiag_cor_Lin = [mean(corr_ensemble_Lin[id_observe, id_offdia, id_cor,:]) for id_observe in 1:n_observe_set]
corr_offdiag_cor_Bez = [mean(corr_ensemble_Bez[id_observe, id_offdia, id_cor,:]) for id_observe in 1:n_observe_set];

corr_diag_cor_Lin_std = [std(corr_ensemble_Lin[id_observe, id_diag, id_cor,:])/sqrt_n_iteration_max for id_observe in 1:n_observe_set]
corr_diag_cor_Bez_std = [std(corr_ensemble_Bez[id_observe, id_diag, id_cor,:])/sqrt_n_iteration_max for id_observe in 1:n_observe_set]
corr_offdiag_cor_Lin_std = [std(corr_ensemble_Lin[id_observe, id_offdia, id_cor,:])/sqrt_n_iteration_max for id_observe in 1:n_observe_set]
corr_offdiag_cor_Bez_std = [std(corr_ensemble_Bez[id_observe, id_offdia, id_cor,:])/sqrt_n_iteration_max for id_observe in 1:n_observe_set];


### Output: slope and Pearson correlation values

In [36]:
fout_Lin_corr = open(dir_out * "sampling_interval_dependency_Lin_corr.dat", "w") 
fout_Bez_corr = open(dir_out * "sampling_interval_dependency_Bez_corr.dat", "w") 
fout_Lin_slope = open(dir_out * "sampling_interval_dependency_Lin_slope.dat", "w") 
fout_Bez_slope = open(dir_out * "sampling_interval_dependency_Bez_slope.dat", "w") ;

for n in 1:size(corr_diag_slope_Lin,1)
    println(fout_Lin_slope, t_observe_set[n], "\t", corr_diag_slope_Lin[n], "\t", corr_diag_slope_Lin_std[n], "\t", corr_offdiag_slope_Lin[n], "\t", corr_offdiag_slope_Lin_std[n])
    println(fout_Bez_slope, t_observe_set[n], "\t", corr_diag_slope_Bez[n], "\t", corr_diag_slope_Bez_std[n], "\t", corr_offdiag_slope_Bez[n], "\t", corr_offdiag_slope_Bez_std[n])
    
    println(fout_Lin_corr, t_observe_set[n], "\t", corr_diag_cor_Lin[n], "\t", corr_diag_cor_Lin_std[n], "\t", corr_offdiag_cor_Lin[n], "\t", corr_offdiag_cor_Lin_std[n])
    println(fout_Bez_corr, t_observe_set[n], "\t", corr_diag_cor_Bez[n], "\t", corr_diag_cor_Bez_std[n], "\t", corr_offdiag_cor_Bez[n], "\t", corr_offdiag_cor_Bez_std[n])
end
close(fout_Lin_slope); close(fout_Bez_slope); close(fout_Lin_corr); close(fout_Bez_corr);
    

### Output: linear regrssion parameters 

In [37]:
reg_coef_diag_Lin_slope = linreg_tot(t_observe_set, corr_diag_slope_Lin)
reg_coef_offdiag_Lin_slope = linreg_tot(t_observe_set, corr_offdiag_slope_Lin)
reg_coef_diag_Bez_slope = linreg_tot(t_observe_set, corr_diag_slope_Bez)
reg_coef_offdiag_Bez_slope = linreg_tot(t_observe_set, corr_offdiag_slope_Bez);
    
reg_coef_diag_Lin_corr = linreg_tot(t_observe_set, corr_diag_cor_Lin)
reg_coef_offdiag_Lin_corr = linreg_tot(t_observe_set, corr_offdiag_cor_Lin);
reg_coef_diag_Bez_corr = linreg_tot(t_observe_set, corr_diag_cor_Bez)
reg_coef_offdiag_Bez_corr = linreg_tot(t_observe_set, corr_offdiag_cor_Bez);


fout_Lin_linreg = open(dir_out * "linreg-parameters_Lin.dat", "w") 
fout_Bez_linreg = open(dir_out * "linreg-parameters_Bez.dat", "w") ;

#Linear: diag_slope, off-diag_slope, diag_cor, off-diag_cor
println(fout_Lin_linreg, reg_coef_diag_Lin_slope[1], "\t", reg_coef_offdiag_Lin_slope[1], "\t", reg_coef_diag_Lin_corr[1], "\t", reg_coef_offdiag_Lin_corr[1])
println(fout_Lin_linreg, reg_coef_diag_Lin_slope[2], "\t", reg_coef_offdiag_Lin_slope[2], "\t", reg_coef_diag_Lin_corr[2], "\t", reg_coef_offdiag_Lin_corr[2])

#Bezier: diag_slope, off-diag_slope, diag_cor, off-diag_cor
println(fout_Bez_linreg, reg_coef_diag_Bez_slope[1], "\t", reg_coef_offdiag_Bez_slope[1], "\t", reg_coef_diag_Bez_corr[1], "\t", reg_coef_offdiag_Bez_corr[1])
println(fout_Bez_linreg, reg_coef_diag_Bez_slope[2], "\t", reg_coef_offdiag_Bez_slope[2], "\t", reg_coef_diag_Bez_corr[2], "\t", reg_coef_offdiag_Bez_corr[2])

close(fout_Lin_linreg); close(fout_Bez_linreg);